In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import math
import datetime
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn import tree

In [3]:
training_df = pd.read_csv("flights_train.csv")

In [4]:
training_y = training_df['ARR_DELAY']

In [33]:
day_of_year = [datetime.datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_yday for date_str in training_df['FL_DATE'] ]
month = [datetime.datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_mon for date_str in training_df['FL_DATE'] ]
training_x = training_df.drop(['UID','FL_NUM','AIRLINE_ID','ORIGIN_CITY_NAME','DISTANCE', 'FL_DATE', 'DEST_CITY_NAME','ORIGIN_STATE_ABR','DEST_STATE_ABR','ARR_DELAY', ], 
                 axis=1)  
training_x['month'] = month
training_x['day_of_year'] = day_of_year
carrier_ohe = pd.get_dummies(training_x['UNIQUE_CARRIER'])
training_x = training_x + carrier_ohe

In [34]:

training_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4911 entries, 0 to 4910
Data columns (total 26 columns):
AA                       0 non-null float64
ACTUAL_ELAPSED_TIME      0 non-null float64
AS                       0 non-null float64
B6                       0 non-null float64
CRS_DEP_TIME             0 non-null float64
DAY_OF_WEEK              0 non-null float64
DEST                     0 non-null object
DEST_CITY_MARKET_ID      0 non-null float64
DISTANCE_GROUP           0 non-null float64
DL                       0 non-null float64
EV                       0 non-null float64
F9                       0 non-null float64
FIRST_DEP_TIME           0 non-null object
HA                       0 non-null float64
NK                       0 non-null float64
OO                       0 non-null float64
ORIGIN                   0 non-null object
ORIGIN_CITY_MARKET_ID    0 non-null float64
TAXI_IN                  0 non-null float64
TAXI_OUT                 0 non-null float64
UA              

In [20]:
x_test = training_x.drop(['UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'FIRST_DEP_TIME'], axis=1)
X_new = SelectKBest(mutual_info_regression, k=3).fit_transform(x_test, training_y)
X_new[0:5]

/Users/lauraedelson/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


array([[1020,   11,   11],
       [1220,   13,    5],
       [ 945,   48,   11],
       [1437,   17,    4],
       [ 830,   37,    3]])

In [11]:
x_test[0:5]

,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE_GROUP,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,month,day_of_year
0,2,1020,11,8,373,11,30559,32982,1,3
1,6,1220,13,9,183,5,33570,30194,1,28
2,6,945,48,6,355,11,30299,30466,1,21
3,1,1437,17,8,136,4,32457,30325,1,23
4,1,830,37,7,114,3,30397,30852,1,2


In [27]:
rfe = RFE(SVR(kernel="linear"), 4)
fit = rfe.fit(x_test, training_y)

KeyboardInterrupt: 

In [25]:
fit.ranking_

array([1, 5, 2, 1, 4, 1, 6, 7, 1, 3])

In [37]:
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(training_x, training_y)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').